In [15]:
import os
from pathlib import Path
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

In [ ]:
# set goofle api key in os.env 
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")  
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")  

os.environ['GOOGLE_API_KEY'] =  GOOGLE_API_KEY
os.environ['LANGCHAIN_API_KEY'] =  LANGCHAIN_API_KEY
os.environ['LANGCHAIN_PROJECT'] =  LANGCHAIN_PROJECT

In [5]:
# Reading PDF file from resources 
file_path = Path(r"D:\SHUBHAM\git_repo\ask-your-pdf\resources\Shubham_Karale_DS_2025_v1.pdf")
pdf_reader= PdfReader(file_path)

In [6]:
# extract all text from pdf 
text=""

for page in pdf_reader.pages:
    # print(page.extract_text())
    text = text+page.extract_text()

print(text)

Shubham Karale 
Data Scientist 
Experienced Data Scientist with 
7 years of experience
, including 
3 years specializing in Data Science & MLOps
.
Currently at 
Capgemini
, optimizing ML models and 
migrating SAS-based ETL pipelines to PySpark
, reducing model
inference time by 
30%
 and improving accuracy by 
15%
. Proﬁcient in 
Python, PySpark, Jenkins, Cloudera, AWS, and
ML model testing
. Strong expertise in 
feature engineering, hyperparameter tuning, cloud deployment, and
MLOps pipelines
. Seeking to leverage expertise in data-driven solutions and cloud-based machine learning. 
skarale63@gmail.com 
+91 96896 49778 
pune, India 
linkedin.com/in/shubham-karale-b343a7221 
WORK EXPERIENCE 
Data Scientist 
Capgemini Technology Services India Limited 
05/2024 - Present
, 
 
Migrated 
SAS-based data pipelines
 to 
PySpark
, optimizing data extraction for ML
models. 
Extracted only relevant features
 from legacy SAS pipelines, reducing model
inference time by 
30%
 and improving accuracy

In [7]:
# Text convert into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
chunks = text_splitter.split_text(text)
chunks

['Shubham Karale \nData Scientist \nExperienced Data Scientist with \n7 years of experience\n, including \n3 years specializing in Data Science & MLOps\n.\nCurrently at \nCapgemini\n, optimizing ML models and \nmigrating SAS-based ETL pipelines to PySpark\n, reducing model\ninference time by \n30%\n and improving accuracy by \n15%\n. Proﬁcient in \nPython, PySpark, Jenkins, Cloudera, AWS, and\nML model testing\n. Strong expertise in \nfeature engineering, hyperparameter tuning, cloud deployment, and\nMLOps pipelines\n. Seeking to leverage expertise in data-driven solutions and cloud-based machine learning. \nskarale63@gmail.com \n+91 96896 49778 \npune, India \nlinkedin.com/in/shubham-karale-b343a7221 \nWORK EXPERIENCE \nData Scientist \nCapgemini Technology Services India Limited \n05/2024 - Present\n, \n \nMigrated \nSAS-based data pipelines\n to \nPySpark\n, optimizing data extraction for ML\nmodels. \nExtracted only relevant features\n from legacy SAS pipelines, reducing model\ninf

In [10]:
# apply embedding and store in faiss
embeddings = OllamaEmbeddings()
vector_store = FAISS.from_texts(chunks, embedding=embeddings)

vector_store

In [ ]:
# use LLM model for chat
llm=Ollama()
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vector_store.as_retriever(), memory=memory)


C:\Users\S Karale\AppData\Local\Temp\ipykernel_18188\2480747478.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama()
C:\Users\S Karale\AppData\Local\Temp\ipykernel_18188\2480747478.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)
